In [5]:
import os
import sys
import time

import matplotlib.pyplot as plt

%matplotlib inline
import numpy as np
import pandas as pd
from IPython.display import HTML
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
sys.path.append(os.path.join(os.path.abspath(".."), "code"))
from sklearn.feature_extraction.text import CountVectorizer
import mglearn
from IPython.display import display
from sklearn.linear_model import LogisticRegression
# Classifiers and regressors
from sklearn.dummy import DummyClassifier, DummyRegressor

# Preprocessing and pipeline
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
# train test split and cross validation
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.pipeline import Pipeline

%matplotlib inline
pd.set_option("display.max_colwidth", 200)
DATA_DIR = "../data/"

In [6]:
df = pd.read_csv('./Phishing_Email.csv')
df.head()
df.shape

(18650, 3)

In [7]:
scoring_metric = 'accuracy'
def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
    from sklearn.model_selection import cross_validate
    import numpy as np
    import pandas as pd

    # Ensure return_train_score is set if not already provided
    if 'return_train_score' not in kwargs:
        kwargs['return_train_score'] = True

    results = cross_validate(model, X_train, y_train, **kwargs)
    print(f"Mean validation score: {np.mean(results['test_score']):.3f}")
    print(f"Mean training score: {np.mean(results['train_score']):.3f}")

In [8]:
import re

def replace_tags(doc):
    doc = doc.replace("<br />", " ")
    doc = re.sub("https://\S*", "", doc)
    return doc